In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder,StandardScaler

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer 
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

from tqdm.notebook import tqdm

In [3]:
stat = pd.read_parquet(r'C:\Users\Gimly\Desktop\data sience\date\traindsfd')

In [4]:
# 1. Отношение лайков к дизлайкам
stat['like_to_dislike_ratio'] = stat['v_likes'] / (stat['v_dislikes'] + 1)  # +1 чтобы избежать деления на ноль

# 2. Отношение просмотров к лайкам и дизлайкам
stat['views_to_likes_ratio'] = stat['v_year_views'] / (stat['v_likes'] + 1)
stat['views_to_dislikes_ratio'] = stat['v_year_views'] / (stat['v_dislikes'] + 1)

# 3. Отношение продолжительности просмотра к продолжительности видео
stat['watchtime_to_duration_1_day'] = stat['v_avg_watchtime_1_day'] / stat['v_duration']
stat['watchtime_to_duration_7_day'] = stat['v_avg_watchtime_7_day'] / stat['v_duration']
stat['watchtime_to_duration_30_day'] = stat['v_avg_watchtime_30_day'] / stat['v_duration']

# 4. Суммарные показатели за разные периоды
stat['total_views'] = stat['v_year_views'] + stat['v_month_views'] + stat['v_week_views'] + stat['v_day_views']
stat['total_likes'] = stat['v_likes']
stat['total_dislikes'] = stat['v_dislikes']
stat['total_long_views'] = stat['v_long_views_1_days'] + stat['v_long_views_7_days'] + stat['v_long_views_30_days']

# 5. Средние показатели за разные периоды
stat['avg_likes_per_day'] = stat['v_likes'] / 365
stat['avg_dislikes_per_day'] = stat['v_dislikes'] / 365
stat['avg_views_per_day'] = stat['v_year_views'] / 365

In [5]:
cat = LabelEncoder()
stat['category_id'] = cat.fit_transform(stat['category_id'])

In [6]:
for col in stat.columns:
    if len(Counter(stat[col]))==1: 
        stat.drop(columns=[col], inplace=True)

In [7]:
stat = stat.drop(columns = ['v_pub_datetime','author_id','title','description'])

In [8]:
#stat = stat.drop(columns = ['v_cr_click_comment_1_days','v_cr_click_vtop_1_days','v_cr_click_comment_1_days','v_cr_click_dislike_1_days','v_cr_click_like_1_days','v_cr_click_comment_30_days','v_cr_click_vtop_30_days','v_cr_click_dislike_30_days','v_cr_click_like_30_days','v_cr_click_comment_7_days','v_cr_click_vtop_7_days','v_cr_click_dislike_7_days','v_cr_click_like_7_days'])

In [9]:
# stat.loc[:, ~stat.columns.isin(['video_id'])]

In [10]:
train =pd.read_parquet(r'C:\Users\Gimly\Desktop\data sience\date\tr.parquet')
train.head(5)

,event_timestamp,user_id,region,city,video_id,watchtime
0,2024-08-10 20:13:12+03:00,73dadec9-9189-43a8-a32f-906e549af7e0,13925c13-0e77-4548-acdc-09aa728e31d4,faeecbed-060a-4d40-be81-ce3bac90804c,98380888-2d8a-4d2f-9660-a00c3c5c0512,0
1,2024-08-10 08:43:24+03:00,314f3f16-d606-4b19-a565-5197f35b4297,d7c6be26-f823-4c5e-83fc-c1446ef917a9,e8856e8a-e3db-4780-843c-b18d1426f0b8,bc1d8afb-ed43-41da-a7e3-40d195b1295b,304
2,2024-08-10 14:31:13+03:00,d8ab4c2f-42e8-4d86-861d-3ad7f1a0e976,243942ba-d0ce-4753-9432-3ed1ba4ae70b,f5343b37-2438-43ac-b98a-b984a38858ef,80a741ad-d2be-445e-90ad-80b345ef8697,0
3,2024-08-10 15:26:14+03:00,874efe8e-eee2-41a6-97bc-3ef0f6d87276,7d0eb6c0-242f-404b-a862-bc6630ddb4b3,78d7f303-b196-4da1-b893-9928c424d20a,34de4313-608a-4bdf-8e7b-6afa2c333247,0
4,2024-08-10 23:10:33+03:00,6478e986-4691-47f9-a0a6-7051550abadc,7d0eb6c0-242f-404b-a862-bc6630ddb4b3,78d7f303-b196-4da1-b893-9928c424d20a,5d2918ad-e728-4a94-a516-d2d91532c9f9,690


In [11]:
test = pd.read_parquet(r'C:\Users\Gimly\Desktop\data sience\date\testasda')
test.head(3)

,event_timestamp,user_id,region,city,video_id
0,2024-08-10 11:45:52+03:00,00000a62-336d-4b71-bb8b-991c877ac678,e7a113d7-5be1-44c0-9e97-78fd27df336f,7a193a03-5883-4ef2-b69d-873f397ec104,62a44a87-29e5-464d-a9ff-78de16d4a7d7
1,2024-08-10 11:53:05+03:00,00000a62-336d-4b71-bb8b-991c877ac678,e7a113d7-5be1-44c0-9e97-78fd27df336f,7a193a03-5883-4ef2-b69d-873f397ec104,a9aa2265-4d43-48e6-aae7-4b5e3f5eb538
2,2024-08-10 12:00:16+03:00,00000a62-336d-4b71-bb8b-991c877ac678,e7a113d7-5be1-44c0-9e97-78fd27df336f,7a193a03-5883-4ef2-b69d-873f397ec104,322aafe1-127c-42d2-aada-bc3a07d612bf


In [12]:
train = train.drop(columns = ['event_timestamp','region','city'])
test = test.drop(columns = ['event_timestamp','region','city'])

In [13]:
def is_long_view(row):
    if row['v_duration'] > 300:  # Видео более 5 минут
        return 1 if row['watchtime'] > (row['v_duration'] * 0.25) else 0
    elif row['v_duration'] <= 300 :  # Видео менее или равно 5 минут
        return 1 if row['watchtime'] > 30 else 0
    else:
        return 0

In [14]:
train = pd.merge(train, stat, on='video_id', how='left')

In [15]:
test = pd.merge(test, stat, on='video_id', how='left')

In [16]:
combined_vid = pd.concat([train['video_id'], test['video_id']]).unique()
combined_us = pd.concat([train['user_id'], test['user_id']]).unique()

In [17]:
v_e = LabelEncoder()
v_e.fit(combined_vid)

LabelEncoder()

In [18]:
u_e = LabelEncoder()
u_e.fit(combined_us)

LabelEncoder()

In [19]:
combined_us = None
combined_vid = None

In [20]:
train['user_id'] = u_e.transform(train['user_id'] )
train['video_id'] = v_e.transform(train['video_id'] )

In [21]:
test['user_id'] = u_e.transform(test['user_id'] )
test['video_id'] = v_e.transform(test['video_id'] )

In [22]:
# Применение функции к колонке 'duration'
train['is_long_view'] = train.apply(is_long_view, axis=1)

In [23]:
x = train.drop(['watchtime','is_long_view'],axis=1)
y = train['is_long_view']

In [24]:
x_train,x_val,y_train,y_val = train_test_split(x, y, test_size=0.2, stratify=y)

In [25]:
stat = None
train = None

In [32]:
model = CatBoostClassifier(task_type = 'GPU', random_state = 42 , iterations = 20000, depth = 15)

In [33]:
model.fit(x_train,y_train, eval_set=(x_val,y_val), verbose=200)

Learning rate set to 0.009318
0:	learn: 0.6900509	test: 0.6900581	best: 0.6900581 (0)	total: 716ms	remaining: 3h 58m 40s
200:	learn: 0.5552453	test: 0.5555742	best: 0.5555742 (200)	total: 2m 1s	remaining: 3h 19m 39s
400:	learn: 0.5501622	test: 0.5510157	best: 0.5510157 (400)	total: 4m 4s	remaining: 3h 19m 26s
600:	learn: 0.5484575	test: 0.5500162	best: 0.5500162 (600)	total: 6m 8s	remaining: 3h 18m 9s
800:	learn: 0.5471646	test: 0.5494802	best: 0.5494802 (800)	total: 8m 13s	remaining: 3h 17m 7s
1000:	learn: 0.5459088	test: 0.5490862	best: 0.5490862 (1000)	total: 10m 20s	remaining: 3h 16m 22s
1200:	learn: 0.5447805	test: 0.5488120	best: 0.5488120 (1200)	total: 12m 30s	remaining: 3h 15m 41s
1400:	learn: 0.5437318	test: 0.5486104	best: 0.5486104 (1400)	total: 14m 40s	remaining: 3h 14m 50s
1600:	learn: 0.5427639	test: 0.5484708	best: 0.5484708 (1600)	total: 16m 50s	remaining: 3h 13m 27s
1800:	learn: 0.5418131	test: 0.5483756	best: 0.5483756 (1800)	total: 19m	remaining: 3h 12m 7s
2000:	lear

In [34]:
score = model.predict(test)

In [29]:
subm = pd.read_csv(r'C:\Users\Gimly\Desktop\data sience\date\id_567__3d3a76ed3a49880b74bd7fbbf597681a_GYhysqc.csv')

In [35]:
subm['target'] = score

In [36]:
subm.to_csv('sub12331221332121ccccccccccccccccc.csv',index = False)